In [1]:
import datetime
import logging
import time

import requests
import json

from web3 import Web3


logging.basicConfig(level=logging.DEBUG)

In [2]:
infura_proj_id = '1be1e9b98a26430582d7793fc5936215'
w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/{}'.format(infura_proj_id)))
w3.isConnected()

DEBUG:web3.providers.HTTPProvider:Making request HTTP. URI: https://mainnet.infura.io/v3/1be1e9b98a26430582d7793fc5936215, Method: web3_clientVersion
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): mainnet.infura.io:443
DEBUG:urllib3.connectionpool:https://mainnet.infura.io:443 "POST /v3/1be1e9b98a26430582d7793fc5936215 HTTP/1.1" 200 94
DEBUG:web3.providers.HTTPProvider:Getting response HTTP. URI: https://mainnet.infura.io/v3/1be1e9b98a26430582d7793fc5936215, Method: web3_clientVersion, Response: {'jsonrpc': '2.0', 'id': 0, 'result': 'Geth/v1.10.15-omnibus-hotfix-f4decf48/linux-amd64/go1.17.6'}


True

In [3]:
addr = '0x7Be8076f4EA4A4AD08075C2508e481d6C946D12b'
url = 'http://api.etherscan.io/api?module=contract&action=getabi&address=0x7Be8076f4EA4A4AD08075C2508e481d6C946D12b'
resp = requests.get(url)
contract_ABI = resp.json()['result']
contract_ABI = json.loads(contract_ABI)

web3_contract = w3.eth.contract(
        abi=contract_ABI,
        address=addr)
block_filter = web3_contract.events.OrdersMatched.createFilter(fromBlock='latest')

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.0.1:4780
DEBUG:urllib3.connectionpool:http://127.0.0.1:4780 "GET http://api.etherscan.io/api?module=contract&action=getabi&address=0x7Be8076f4EA4A4AD08075C2508e481d6C946D12b HTTP/1.1" 200 2246
DEBUG:web3.RequestManager:Making request. Method: eth_newFilter
DEBUG:web3.providers.HTTPProvider:Making request HTTP. URI: https://mainnet.infura.io/v3/1be1e9b98a26430582d7793fc5936215, Method: eth_newFilter
DEBUG:urllib3.connectionpool:https://mainnet.infura.io:443 "POST /v3/1be1e9b98a26430582d7793fc5936215 HTTP/1.1" 200 82
DEBUG:web3.providers.HTTPProvider:Getting response HTTP. URI: https://mainnet.infura.io/v3/1be1e9b98a26430582d7793fc5936215, Method: eth_newFilter, Response: {'jsonrpc': '2.0', 'id': 1, 'result': '0x10ff0e81e231f9e99c8ae8b538a0cf3afc64489af4d3'}


In [4]:
contract_instant_deal_kv = {}


def log_topk(contract_instant_deal_kv, k=10):
    hot_contracts = sorted(contract_instant_deal_kv.items(), key=lambda x: x[1]['deal_num'], reverse=True)[:10]
    logging.info('\nHot top 10 contract in {}:'.format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())))
    for contract in hot_contracts:
        logging.info(contract)
    

def looping(event_filter, poll_interval):
    while True:
        for event in event_filter.get_new_entries():
            tx_value_eth, tx_from, tx_to, tx_datetime, addr = handle_event(event)
            try:
                contract_instant_deal_kv[addr]['deal_num'] += 1
                contract_instant_deal_kv[addr]['eth_amount'] += tx_value_eth
            except KeyError:
                contract_instant_deal_kv[addr] = {'deal_num': 1, 'eth_amount': tx_value_eth}
        time.sleep(poll_interval)
        log_topk(contract_instant_deal_kv)

In [5]:
def timestamp_to_datetime(timeStamp):
    timeArray = time.localtime(timeStamp)
    otherStyleTime = time.strftime("%Y-%m-%d %H:%M:%S", timeArray)
    return otherStyleTime


def handle_event(event):
    transactionHash = event['transactionHash']
    tx = w3.eth.get_transaction(transactionHash)
    tx_value_wei = tx['value']
    tx_value_eth = Web3.fromWei(tx_value_wei, 'ether')
    tx_from = tx['from']
    tx_to = tx['to']
    # 获取去块时间
    tx_block_number = tx['blockNumber']
    tx_timestamp = w3.eth.getBlock(tx_block_number).timestamp
    tx_datetime = timestamp_to_datetime(tx_timestamp)
    addr = get_contract_addr(tx['input'])
    return tx_value_eth, tx_from, tx_to, tx_datetime, addr

In [6]:
def get_contract_addr(input_str):
    try:
        input_decoded = web3_contract.decode_function_input(input_str)
    except Exception:
        return ''
    input_decoded = input_decoded[1]
    addrs = input_decoded['addrs']
    addr4 = addrs[4].lower()
    addr11 = addrs[11].lower()
    assert addr4 == addr11
    return addr4

In [52]:
# read filter entries every 2 sec
looping(block_filter, 2)

Hot top 10 contract in 2022-01-28 01:17:25:
('0xd532b88607b1877fe20c181cba2550e3bbd6b31c', {'deal_num': 46, 'eth_amount': Decimal('54.488')})
('', {'deal_num': 13, 'eth_amount': Decimal('26.95')})
('0x236672ed575e1e479b8e101aeeb920f32361f6f9', {'deal_num': 6, 'eth_amount': Decimal('2.193')})
('0xbabe3dec32fa870b4125b9cad1cea498dcb116c9', {'deal_num': 3, 'eth_amount': Decimal('0.780')})
('0x40a0f793ccfd9bc6f2532d06d022be0f978ecbf3', {'deal_num': 2, 'eth_amount': Decimal('0.4')})
('0x9ec46a3dd227fd5977c7f1997c043122a703ff2a', {'deal_num': 2, 'eth_amount': Decimal('0.54')})
('contract_addr', {'deal_num': 1, 'eth_amount': 0.1})
('0x496299d8497a02b01f5bc355298b0a831f06c522', {'deal_num': 1, 'eth_amount': Decimal('0.135')})
('0x65cc7530e8c6f5a51257f7b7586361c4a22cec93', {'deal_num': 1, 'eth_amount': Decimal('0.368')})
('0xba30e5f9bb24caa003e9f2f0497ad287fdf95623', {'deal_num': 1, 'eth_amount': Decimal('8.5')})

Hot top 10 contract in 2022-01-28 01:19:24:
('0xd532b88607b1877fe20c181cba2550e3b

ProxyError: HTTPSConnectionPool(host='mainnet.infura.io', port=443): Max retries exceeded with url: /v3/1be1e9b98a26430582d7793fc5936215 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(54, 'Connection reset by peer')))

In [30]:
"""
contract addr example
"""

transaction_id = '0xc326b17e9a6b6df3cb019e1a64d0ccd42a3ddb3f084f6c1d2a9cbb4bf3d30b34'

tx = w3.eth.get_transaction(transaction_id)
print(list(tx.keys()))
input_str = tx['input']

input_decoded = web3_contract.decode_function_input(input_str)
input_decoded = input_decoded[1]
addrs = input_decoded['addrs']
addr4 = addrs[4]
addr11 = addrs[11]
print(addr4, addr11)

0x496299d8497A02b01f5Bc355298b0a831f06c522 0x496299d8497A02b01f5Bc355298b0a831f06c522
